In [ ]:
import numpy as np
import math
import random

In [ ]:
#Función que genera un .txt con tantas líneas como se indique donde cada línea tendrá tantos números complejos como se indique (por defecto, 1000 valores por linea,
#que es la cantidad de valores en 20ms con sample rate 50k)
def generate_and_save_complex_numbers(file_path, num_lines, num_complex_numbers_per_line=1000, type='random'):
  with open(file_path, "w") as file:

    if type == 'random': # Se define el archivo con num_lines lineas, cada una con num_complex_numbers_per_line valores aleatorios.
      for line_number in range(0, num_lines):
        complex_numbers = [np.round(complex(random.uniform(-1*0.001, 1*0.001), random.uniform(-1*0.001, 1*0.001)), decimals=3) for _ in range(num_complex_numbers_per_line)]
        for num in complex_numbers:
            file.write(f"{num}\n")
        #file.write("\n")

    else:
      if type == 'probar20ms1': # Se define el archivo con num_lines lineas, cada una con 1000 muestras (correspondientes a 20 ms). Cada subintervalo de 20ms
                             # tendra un valor 0.1 mayor al anterior (en ambas componentes, compleja y real),
                             # iniciando en cero. Nos interesa probar que se calcula de forma correcta el promedio
                             # en cada subintervalo de 20 ms
        ant = 0
        for line_number in range(0, num_lines):
          complex_numbers = [np.round(complex(0.001+ant, 0.001+ant), decimals=3) for _ in range(num_complex_numbers_per_line)]
          for num in complex_numbers:
            file.write(f"{num}\n")
            ### primero probamos con todos los valores de 20ms iguales, y luego podemos tener los primeros 10ms de un subintervalo con cierto valor, y los
            ### segundos 10ms de otro valor, esto para probar que los promedios intervalos
          ant = ant + 0.001

      else:
        if type == 'probar20ms2': # para tener en cada intervalo de 20ms con unos primeros 10ms de un valor X y los segundos 10ms de un valor Y
          ant = 0
          for line_number in range(0, num_lines):
            complex_numbers = [np.round(complex(0.001+ant, 0.001+ant), decimals=3) for _ in range(int(num_complex_numbers_per_line/2))]
            for num in complex_numbers:
              file.write(f"{num}\n")
            complex_numbers = [np.round(complex(0.002+ant, 0.002+ant), decimals=3) for _ in range(int(num_complex_numbers_per_line/2))]
            for num in complex_numbers:
              file.write(f"{num}\n")

            ant = ant + 0.002
        else:
          if type == 'probar4s1': # cada subintervalo de 20 ms tiene siempre el mismo valor, y en los 200 subintervalos, los valores van aumentando
            ant = 0
            for line_number in range(0, num_lines):
              complex_numbers = [np.round(complex(0.001+ant, 0.001+ant), decimals=3) for _ in range(num_complex_numbers_per_line)]
              for num in complex_numbers:
                file.write(f"{num}\n")
              ant = ant + 0.001
          else:
            if type == 'probar4s2': # cada subintervalo de 20 ms tiene siempre el mismo valor, pero estos valores fijos van disminuyendo en los primeros 300
                                    # subintervalos, y luego van aumentando.

              ant = 0.001*800 #valor inicial, que es maximo, porque vamos disminuyendo
              for line_number in range(0, int(num_lines/2)):
                complex_numbers = [np.round(complex(ant-0.001, ant-0.001), decimals=3) for _ in range(num_complex_numbers_per_line)]
                for num in complex_numbers:
                  file.write(f"{num}\n")
                ant = ant-0.001
              for line_number in range(0, int(num_lines/2)):
                complex_numbers = [np.round(complex(ant+0.001, ant+0.001), decimals=3) for _ in range(num_complex_numbers_per_line)]
                for num in complex_numbers:
                  file.write(f"{num}\n")
                ant = ant+0.001
            else:
              if type == 'probar1min':
                ant = 0.001
                for line_number in range(0, num_lines):
                  if line_number % 200 == 0 and line_number != 0:
                    ant = ant + 0.001 # cada 200 intervalos de 20 ms, que son 4 s, cambio el valor. Esto es,
                                      # todos los intervalos de 4 s tienen todos los valores iguales
                  complex_numbers = [np.round(complex(ant, ant), decimals=3) for _ in range(num_complex_numbers_per_line)]
                  for num in complex_numbers:
                    file.write(f"{num}\n")
              else:
                if type == 'probar1min_2':
                  ant = 0.001
                  for line_number in range(0, int(num_lines/2)):
                    if line_number % 200 == 0 and line_number != 0:
                      ant = ant + 0.001 # cada 200 intervalos de 20 ms, que son 4 s, cambio el valor. Esto es,
                                      # todos los intervalos de 4 s tienen todos los valores iguales
                    complex_numbers = [np.round(complex(ant, ant), decimals=3) for _ in range(num_complex_numbers_per_line)]
                    for num in complex_numbers:
                      file.write(f"{num}\n")

                  for line_number in range(0, int(num_lines/2)):
                    if line_number % 200 == 0 and line_number != 0:
                      ant = ant - 0.001 # cada 200 intervalos de 20 ms, que son 4 s, cambio el valor. Esto es,
                                      # todos los intervalos de 4 s tienen todos los valores iguales
                    complex_numbers = [np.round(complex(ant, ant), decimals=3) for _ in range(num_complex_numbers_per_line)]
                    for num in complex_numbers:
                      file.write(f"{num}\n")

### Función para calcular la potencia actual y el umbral

In [ ]:
def potumbral_muestra(muestra_actual, umbral, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N):
  power = np.abs(muestra_actual)**2 #Paso la muestra actual compleja a un real con la potencia del complejo. Esto permite acumular
                                    #las muestras por un periodo de tiempo arbitrario (que definimos como 1ms) y luego tomar un promedio
                                    #para obtener la potencia atual del canal.

  #Para tomar el promedio de la potencia actual en el canal se toma un "promedio movil". En el for se rotan los valores del arreglo un
  #lugar para la derecha excepto el primero que se va a sobreescribir. Si por ejemplo en el arreglo teniamos arr=[1, 2, 3, 4] despues
  #del for va a quedar arr=[1,1,2,3]. Después se sobreescribe el primer elemento y se vuelve a calcular el promedio.
  for i in range(len(arr_pow_actual) - 1, 0, -1):
    arr_pow_actual[i] = arr_pow_actual[i - 1]

  arr_pow_actual[0] = power

  pow_actual = np.mean(arr_pow_actual) #La potencia actual del canal es el promedio de la potencia del ultimo ms del canal

  umbral_actual = (10*math.log10(umbral)) + 10 #Se le suma 10dB al umbral, el umbral actual es el ultimo calculado que se actualiza cada 4s.

  umb_y_pow_actual = np.zeros(2)
  umb_y_pow_actual[0] = umbral_actual
  umb_y_pow_actual[1] = pow_actual

  salida = umb_y_pow_actual #Se setea el parametro que se utilizara como salida, se saca siempre un arreglo con 2 elementos, umbral y potencia
                            #actuales (en ese orden).

  N_muestras = np.append(N_muestras, power) #Se acumulan las 1000 muestras para formar los 20ms, esto se acumula independientemente del calculo
                                            #de la potencia actual del canal.
  N = N - 1
  if N == 0: #Si ya acumule la potencia de los ultimos 20 ms.
    pow_avg = np.mean(N_muestras) #Los 200 valores de potencia que se acumulan para formar los 4s es el promedio de los 20ms.
    N_muestras = []
    N = 1000
    pow_4s[subintervalos] = pow_avg #Se acumulan los 200 valores de potencia para llegar a 4s.
    subintervalos += 1
    #print(pow_avg) ### prints utilizados para probar 20 ms
    #print(pow_4s)  ### prints utilizados para probar 20 ms
  if subintervalos == 200: #Si completé los 200 subintervalos de 20ms tengo 4s y agarro el mínimo de eso.
    po_4 = np.min(pow_4s)
    #El umbral del canal se actualiza cada 4s y esto se hace tomando el minimo entre los ultimos 15 valores de 4s de potencia que se acumularon. En otras palabras, en el ultimo minuto se acumulan 15 muestras de potencia de 4s, se toma entonces el minimo de esas 15 como el nuevo valor de umbral. Para actualizar cuales son los ultimos 15 valores de 4s, se hace el mismo procedimiento con el for que con arr_pow_actual, o sea, se shiftea todo el arreglo un lugar para la derecha excepto la primera posicion y se sobreescribe dicho valor.
    for i in range(len(pow_minuto) - 1, 0, -1):
      pow_minuto[i] = pow_minuto[i - 1]

    pow_minuto[0] = po_4
    umbral = np.min(pow_minuto)
    if umbral < 2e-14: #Si la potencia es menor a -107dBm se pone en -107dBm.
      umbral = 2e-14
    elif umbral > 1.9e-4: #Si la potencia es mayor a -7dBm se pone en -7dBm.
      umbral = 1.9e-4
    subintervalos = 0
    print(po_4)
    print(umbral)
    print(pow_minuto)
    print('------')
  return(umbral, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

## Prueba de intervalos de a 20 ms

### Prueba 1

Se prueba en primer lugar que se calcule de forma correcta el promedio de cada intervalo de 20 ms. Cada intervalo de 20 ms tiene siempre el mismo valor (por ejemplo, [0,1; 0,1; 0,1; ...]), con el fin de saber el resultado del promedio para cada uno.

In [ ]:
generate_and_save_complex_numbers("complex_numbers20ms_1.txt", num_lines=3, type='probar20ms1')

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers20ms_1.txt') as f:
    lines = f.readlines()

In [ ]:
# las lineas del .txt quedan definidas como strings. Es necesario pasarlas a
print(len(lines))
largo = len(lines[0])
print(complex(lines[0]))

3000
(0.001+0.001j)


In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

2.0000000000000003e-06
[2.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+

### Prueba 2

Se prueba con intervalos de 20 ms que tienen un valor X en las muestras de los primeros 10 ms, y luego un valor Y en las muestras de los siguientes 10 ms.

In [ ]:
generate_and_save_complex_numbers("complex_numbers20ms_2.txt", num_lines=3, type='probar20ms2')

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers20ms_2.txt') as f:
    lines = f.readlines()

In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

5.000000000000001e-06
[5.e-06 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+0

## Prueba de intervalos de 4 s

### Prueba 1

Prueba de 3 intervalos de 4s, esto es, 3*200 subintervalos de 20 ms. El valor de la potencia va aumentando desde el primer subintervalo hasta el subintervalo numero 600, por lo que el menor valor de cada subintervalo será siempre la primera muestra del mismo.

In [ ]:
generate_and_save_complex_numbers("complex_numbers4s_1.txt", num_lines=600, type='probar4s1')

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers4s_1.txt') as f:
    lines = f.readlines()

In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

2.0000000000000003e-06
2e-14
0.08080200000000001
2e-14
0.3216020000000001
2e-14


Se observa que 2e-6 es la potencia asociada al menor valor de los primeros 200 subintervalos de 20 ms (0.001+0.001j). En los segundos 200 subintervalos se tiene que el primer valor es (0.001+0.001j)*200+(0.001+0.001j), que es el complejo inicial, más el valor de las 200 siguientes iteraciones. La potencia asociada es 0.080802. Finalmente, para el siguiente se considera 0.001+0.001j 401 veces, que tiene la potencia 0.321602.

### Prueba 2
Se prueba con 3 intervalos de 4 s, teniendo 3*200 subintervalos de 20 ms. En los primeros 300 subintervalos la potencia va disminuyendo, y luego comienza a aumentar.

In [ ]:
generate_and_save_complex_numbers("complex_numbers4s_2.txt", num_lines=600, type='probar4s2')

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers4s_2.txt') as f:
    lines = f.readlines()

In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

0.72
2e-14
0.5000000000000001
2e-14
0.7224020000000002
2e-14


Se tiene que el valor más chico en el primer intervalo de 20ms es (0.001+0.001j)x800 - (0.001+0.001j)x200 (pues en los primeros 300 subintervalos se va restando), que es (0.6 + 0.6j). La potencia de este complejo es 0.72.

Para el segundo intervalo de 20ms el valor más chico es de (0.001+0.001j)x800 - (0.001+0.001j)x300 pues se van disminuyendo los valores hasta la mitad del intervalo y después empieza a aumentar. El valor de dicho complejo es (0.005+0.005j) que tiene potencia de 0.5.

Para el tercer intervalo de 20ms, el valor más chico es el primero pues en todos los 200 subintervalos el valor aumenta. Es decir, el valor del complejo más chico en esos 20ms es de (0.001+0.001j)x800 - (0.001+0.001j)x300 + (0.001+0.001j)x100 = (0.001+0.001j)x800 - (0.001+0.001j)x200. Esto da el complejo de (0.6 + 0.6j) cuya potencia es 0.72.

## Prueba de 1 min

### Prueba 1
Los valores de cada intervalo de 4 s van aumentando.

Se busca verificar que se tome el intervalo correcto dentro de los 15 intervalos de 4 s.

In [ ]:
generate_and_save_complex_numbers("complex_numbers1min.txt", num_lines=200*15*3, type='probar1min') # cada linea hace referencia a 20 ms
                                                                                                   # se precisan 200 lineas para 4 s, y 15 veces 4 s para un minuto
                                                                                                   # entonces, 200*15 es un minuto, y fijo 3 minutos para la prueba

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers1min.txt') as f:
    lines = f.readlines()

In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

2.0000000000000003e-06
2e-14
[2.e-06 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14
 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14]
------
8.000000000000001e-06
2e-14
[8.e-06 2.e-06 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14
 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14]
------
1.8e-05
2e-14
[1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
3.2000000000000005e-05
2e-14
[3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
5.0000000000000016e-05
2e-14
[5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
7.2e-05
2e-14
[7.2e-05 5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
9.800000000000001e-05
2e-14
[9.8e-05 7.2e-05 5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14
 2.0e-

### Prueba 2
Se vuelve a probar, pero esta vez a partir de cierto punto los valores de potencia definidos para los intervalos van disminuyendo.

In [ ]:
generate_and_save_complex_numbers("complex_numbers1min_2.txt", num_lines=200*15*3, type='probar1min_2')

In [ ]:
#Cada línea tendrá guardada 1000 muestras que equivalen a 20ms.
with open(r'complex_numbers1min_2.txt') as f:
    lines = f.readlines()

In [ ]:
cantidad = len(lines) # Se carga en "cantidad" la cantidad de líneas
umbral_anterior = 2e-14
subintervalos = 0
N_muestras = []
pow_4s = np.zeros(200)
pow_minuto = np.full(15, 2e-14) #Se inicializa el arreglo para que el umbral de -107dBm=2e-14
arr_pow_actual = np.full(50, 2e-14) #Se inicializa el arreglo que determina la potencia actual con el promedio. Se toma un promedio movil del
                                    #ultimo ms (50 muestras) iniciando con una potencia inicial igual al umbral minimo
N = 1000
for i in range(0, cantidad):
  umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N = potumbral_muestra(complex(lines[i]), umbral_anterior, N_muestras, subintervalos, pow_4s, pow_minuto, arr_pow_actual, N)

2.0000000000000003e-06
2e-14
[2.e-06 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14
 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14]
------
8.000000000000001e-06
2e-14
[8.e-06 2.e-06 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14
 2.e-14 2.e-14 2.e-14 2.e-14 2.e-14]
------
1.8e-05
2e-14
[1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
3.2000000000000005e-05
2e-14
[3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
5.0000000000000016e-05
2e-14
[5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
7.2e-05
2e-14
[7.2e-05 5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14 2.0e-14
 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14 2.0e-14]
------
9.800000000000001e-05
2e-14
[9.8e-05 7.2e-05 5.0e-05 3.2e-05 1.8e-05 8.0e-06 2.0e-06 2.0e-14 2.0e-14
 2.0e-